In [1]:
import tensorflow as tf
from functools import reduce

import numpy as np
import pandas as pd
import os
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 

In [2]:
no_of_channels = 1
image_size     = 28
no_of_labels   = 10
batch_size     = 50 

In [39]:

# #\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
# def get_placeholders_for_images_and_labels():
#     x = tf.placeholder(tf.float32, [None, 28 * 28])
#     y = tf.placeholder(tf.float32, [None, no_of_labels])
    
#     return x, y


x  = tf.placeholder(tf.float32, [None, 28 * 28], name = 'input')
x_reshaped = tf.reshape(x, shape = [-1, 28, 28, 1])
y  = tf.placeholder(tf.float32, [50, no_of_labels], name = 'output')
y_ = tf.reshape(y, [50, 10])



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def get_the_weights_and_biases_for_conv_and_pooling():
    
    weights_1 = tf.Variable(tf.truncated_normal([5, 5, no_of_channels, 32], stddev = 0.2, seed = 23, dtype = tf.float32), name = 'W1')
    bias_1    = tf.Variable(tf.constant(0.1, shape = [32], dtype = tf.float32), name = 'B1')

    weights_2 = tf.Variable(tf.truncated_normal([5, 5, 32, 32], stddev = 0.1, seed = 23, dtype = tf.float32), name = 'W2')
    bias_2    = tf.Variable(tf.constant(0.1, shape = [32], dtype = tf.float32), name = 'B2')
    
    return [[weights_1, bias_1], [weights_2, bias_2]]



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def get_total_no_of_neurons(shape_of_conv_out):
    neurons = 1
    for num in shape_of_conv_out:
        if num is not None:
            neurons = neurons * num
    return neurons



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def get_the_weights_and_biases_for_fc_layer(shape_of_conv_out):
    
    '''multiplying all the elements in the shape list'''
    total_no_of_neurons = get_total_no_of_neurons(shape_of_conv_out)
    
    fc_weights_1 = tf.Variable(tf.truncated_normal([total_no_of_neurons, 1024]), name = 'fc_W1')
    fc_bias_1    = tf.Variable(tf.random_normal([1024]), trainable = True, name = 'fc_B1')
    
    fc_weights_2 = tf.Variable(tf.truncated_normal([1024, no_of_labels]), name = 'fc_W2')
    fc_bias_2    = tf.Variable(tf.random_normal([no_of_labels]), name = 'fc_B2')
    
    return [[fc_weights_1, fc_bias_1], [fc_weights_2, fc_bias_2]]



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def apply_convolution_on_the_data(x, w, b):
    
    convoluted_image = tf.nn.conv2d(x, w, strides = [1, 1, 1, 1], padding = 'SAME')
    CI_with_bias     = tf.nn.bias_add(convoluted_image, b)
    activated_CI     = tf.nn.relu(CI_with_bias)
    return activated_CI



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def apply_max_pooling_on_convolution(convolution, k = 2):
    
    return tf.nn.max_pool(convolution, ksize = [1, k, k, 1], strides = [1, k, k, 1], padding = 'SAME')



#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def CNN():
    
    weights_biases = get_the_weights_and_biases_for_conv_and_pooling()
    w_1, b_1       = weights_biases[0]
    w_2, b_2       = weights_biases[1]
    
    conv_1 = apply_convolution_on_the_data(x_reshaped, w_1, b_1)
    pool_1 = apply_max_pooling_on_convolution(conv_1)
    norm_1 = tf.nn.lrn(pool_1, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)
    
    
    conv_2 = apply_convolution_on_the_data(norm_1, w_2, b_2)
    pool_2 = apply_max_pooling_on_convolution(conv_2)
    norm_2 = tf.nn.lrn(pool_2, depth_radius = 4, bias = 1.0, alpha = 0.002 / 9.0, beta = 0.75)
    
    
    shape_of_norm_2   = norm_2.get_shape().as_list()
    
    fc_weights_biases = get_the_weights_and_biases_for_fc_layer(shape_of_norm_2)
    fc_w_1, fc_b_1    = fc_weights_biases[0]
    fc_w_2, fc_b_2    = fc_weights_biases[1]
    
    norm_2_reshaped = tf.reshape(norm_2, [-1, fc_w_1.get_shape().as_list()[0]])
    fc_layer_1      = tf.add(tf.matmul(norm_2_reshaped, fc_w_1), fc_b_1)
    fc_l_1_relu     = tf.nn.relu(fc_layer_1)
    
    fc_layer_out    = tf.add(tf.matmul(fc_l_1_relu, fc_w_2), fc_b_2)

    return fc_layer_out, [w_1, b_1, w_2, b_2, fc_w_1, fc_b_1, fc_w_2, fc_b_2]


#\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
def get_optimizer_accuracy():
    
    CNN_model, wts_biases   = CNN()
    cost                    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y_, logits = CNN_model))
    train_optim             = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
    correctly_predicted     = tf.equal(tf.argmax(CNN_model, 1), tf.argmax(y_, 1))
    accuracy                = tf.reduce_mean(tf.cast(correctly_predicted, tf.float32))

    return CNN_model, train_optim, accuracy, wts_biases
                        

In [4]:

##########################################################
def read_data(directory_path, is_train_data):
    if is_train_data:
        mnsit_data = pd.read_csv(directory_path, header=None, low_memory = False)
        mnsit_data.drop(0, axis = 0, inplace = True)
        train_Y    = mnsit_data[0].astype(np.float32)
        mnsit_data.drop(0, axis = 1, inplace = True)
        mnsit_data = mnsit_data.astype(np.float32)

        return mnsit_data, train_Y
    
    else:
        mnsit_test = pd.read_csv(directory_path, header = None, low_memory = False)
        mnsit_test.drop(0, axis = 0, inplace = True)
        mnsit_test = mnsit_test.astype(np.float32)
        
        return mnsit_test
    
    
    
##########################################################
def split_the_train_data(train_X, train_Y):
    train_x, train_y, test_x, test_y = train_test_split(train_X, train_Y, test_size = 0.2, 
                                                        random_state = 42)
    
    return [[train_x, test_x], [train_y, test_y]]
    
##########################################################
def convert_labels_to_one_hot_encoding_format(train_y, test_y):
    one_hot_encoder = OneHotEncoder(categories = 'auto')
    train_y_encoded = one_hot_encoder.fit_transform(train_y).toarray()
    test_y_encoded  = one_hot_encoder.fit_transform(test_y).toarray()
    return train_y_encoded.astype(np.float32), test_y_encoded.astype(np.float32)



##########################################################
# def convert_input_to_image_format(row):
#     return row.values.reshape(28, 28)
def convert_input_to_image_format(data):
    output = []
    no_of_rows = np.shape(data)[0]
    for row_num in range(0, no_of_rows):
        output.append(data.iloc[row_num].values.reshape(28, 28, 1))
    
    return np.array(output).astype(np.float32)
    


##########################################################
def standardize_the_data(train_data, test_data):
    scaler = StandardScaler().fit(train_data)
    train_data_scaled = scaler.transform(train_data)
    test_data_scaled  = scaler.transform(test_data)
    return pd.DataFrame(train_data_scaled, dtype = np.float32), pd.DataFrame(test_data_scaled, dtype = np.float32)





In [ ]:
train_X, train_Y = read_data('/Users/vijay/Downloads/digit-recognizer/train.csv', True)
test_X           = read_data('/Users/vijay/Downloads/digit-recognizer/test.csv', False)

train_test_data  = split_the_train_data(train_X, train_Y)
train_x, train_y = train_test_data[0]
test_x, test_y   = train_test_data[1]

train_x_standardized, test_x_standardized = standardize_the_data(train_x, test_x)
# train_x_reshaped = train_x_standardized.apply(convert_input_to_image_format, axis = 1)
# test_x_reshape   = test_x_standardized.apply(convert_input_to_image_format, axis = 1)
train_x_reshaped = convert_input_to_image_format(train_x_standardized)
test_x_reshaped   = convert_input_to_image_format(test_x_standardized)

train_y_encoded, test_y_encoded = convert_labels_to_one_hot_encoding_format(train_y.values.reshape(-1, 1), 
                                                                           test_y.values.reshape(-1, 1))


del(train_x)
del(test_x)
del(train_y)
del(test_y)
del(train_x_standardized)
del(test_x_standardized)

In [40]:

def run_the_CNN_model():
    with tf.Session() as session:
        
        y_out, optimizer, accuracy, wts_biases = get_optimizer_accuracy()
        session.run(tf.global_variables_initializer())
        saver = tf.train.Saver(tf.trainable_variables())
        
        for epoch in range(0, 1):
            for i in range(0, len(train_x_reshaped), batch_size):
                
                train_x = train_x_reshaped[i : i + batch_size, :]
                train_y = train_y_encoded[i : i + batch_size, :]
                _, accuracy_val = session.run([optimizer, accuracy], feed_dict = {x_reshaped : train_x, 
                                                                                  y_ : train_y})
                

            if epoch % 1 == 0:
                print(epoch, accuracy_val)

    
        save_path = saver.save(session, "./trained_model")
        
        return y_out
        

In [ ]:
y_out = run_the_CNN_model()

In [ ]:
#tf.reset_default_graph()
with tf.Session() as session:
    meta_graph = tf.train.import_meta_graph('./trained_model.meta')
    meta_graph.restore(session, tf.train.latest_checkpoint('./'))
    
    prediction = session.run(y_out, feed_dict = {x_reshaped : test_x_reshaped})
    print((session.run(tf.nn.softmax(prediction))))